In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np

In [16]:
## load the trained model,scaler pickle file,onehot
model=load_model("model.h5")

## load the encoder and scaler 
with open("onehot_encoder_geo.pkl","rb") as file:
    lable_encoder_geo= pickle.load(file)

with open("label_encoder_gender.pkl","rb") as file:
    lable_encoder_gender= pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler = pickle.load(file)



In [56]:
#example input
input_data= {
    "CreditScore":600,
    "Geography": "France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance": 6000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [57]:
# one hot encode geography 
geo_encoded= lable_encoder_geo.transform([[input_data["Geography"]]])##.toarray() i am not using .toarray because i have sparsoutput to false in exp ipynb
geo_encoded_df = pd.DataFrame(geo_encoded, columns=lable_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df 

c:\Users\Laxit\Desktop\ANN_Project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [58]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,6000,2,1,1,50000


In [59]:
## Encode catogrical variable
input_df["Gender"]=lable_encoder_gender.transform(input_df["Gender"])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,6000,2,1,1,50000


In [60]:
# Combine onehot encoded colums with input data
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,6000,2,1,1,50000,1.0,0.0,0.0


In [61]:
## scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [62]:
## Pridict the persion is going to leave the bankk or not
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 33ms/step


array([[0.00800197]], dtype=float32)

In [63]:
prediction_prob= prediction[0][0]
prediction_prob

0.008001968

In [64]:
if prediction_prob>0.5:
    print("the customer is likely to churn")
else:
    print("the customer is not likely to churn")

the customer is not likely to churn
